In [ ]:
import re
import os
import pandas as pd
from tqdm.auto import tqdm
!pip install transformers
!pip install datasets
from datasets import Dataset
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_metric
import torch
from sklearn.preprocessing import LabelEncoder
import pickle

     |████████████████████████████████| 2.9 MB 7.5 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 895 kB 29.3 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 23.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 10.5 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
     |████████████████████████████████| 243 kB 51.8 MB/s 
     |████████████████████████████████| 125 kB 52.0 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
     |████████████████████████████████| 160 kB 52.4 MB/s 


In [ ]:
text_dict = {'Train':[], 'Test':[]} 
topics = {'Train':[], 'Test':[]}
path = '/content/drive/MyDrive/reuters21578/' # Path to the folder having the data , Please change these appropriately.
for fname in tqdm([t for t in list(os.walk(path))[0][2] if '.sgm' in t]):
    with open(os.path.join(path,fname),'r',errors='ignore') as f:  # Reading the file 
        data = f.read() 
    data = re.sub('\n|  +',' ',data)  # Removing neline characters and replacing continous multiple spaces to one
    # Spliting the the file on </REUTERS> tag as mentioned by the README
    data = data.split('</REUTERS>')  


    for i,x in tqdm(enumerate(data)):
        if len(x) < 10:
          continue
        reuter_tag = re.findall('<REUTERS (.*?)>',x) # Extracting the <REUTERS> tag
        if 'LEWISSPLIT="TEST"' in reuter_tag[0]: # Using the Lewis Split to get the train and test split
          split = 'Test'
        else:
          split = 'Train'  
        reg_str = "<TOPICS>(.*?)</TOPICS>" # Extracting topic of the article
        res = re.findall(reg_str, x)
        reg_str = "<D>(.*?)</D>"  
        if not res or (not res[0]):
          topics[split].append('N/A') # If no topic is assigned, then assigning N/A for it
        else:  
            result = re.findall(reg_str, res[0])
            topics[split].append(result[0])    # Assigning the first(if multiple found) or the only topic found
        text_type = re.findall('<TEXT (.*?)>',x) # Extracting the text
        if not text_type: # Nominal is Default so no explicit type is defined in the file and article text is between body tags
            reg_str = "<BODY>(.*?)</BODY>"
            article = re.findall(reg_str, x)  
        else:
          if 'TYPE="BRIEF"' in text_type: # If type= brief then the text is in between title tags
              reg_str = "<TITLE>(.*?)</TITLE>"
              article = re.findall(reg_str, x)
          else: # if type = unproc then text is between text tags
              reg_str = '<TEXT TYPE="UNPROC">&#2;(.*?)</TEXT>'
              article = re.findall(reg_str, x)
        if not article:
            print(reuter_tag)
            print(article)
        text_dict[split].append(article[0])      

  0%|          | 0/21 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
# Creating training and testing dataframes

df_train = pd.DataFrame({'Article':text_dict['Train'],  
                         'Topic':topics['Train']})

df_test = pd.DataFrame({'Article':text_dict['Test'],  
                         'Topic':topics['Test']})
print(df_train.shape, df_test.shape)
df_train.head(20)

(15390, 2) (5610, 2)


,Article,Topic
0,Showers continued throughout the week in the B...,cocoa
1,Standard Oil Co and BP North America Inc said ...,N/A
2,Texas Commerce Bancshares Inc's Texas Commerce...,N/A
3,BankAmerica Corp is not under pressure to act ...,N/A
4,The U.S. Agriculture Department reported the f...,grain
5,Argentine grain board figures show crop regist...,veg-oil
6,Red Lion Inns Limited Partnership said it file...,N/A
7,Moody's Investors Service Inc said it lowered ...,N/A
8,Champion Products Inc said its board of direct...,earn
9,Computer Terminal Systems Inc said it has comp...,acq


In [ ]:
df_train['Topic'].value_counts()[:20]

N/A             7045
earn            2888
acq             1708
money-fx         530
grain            410
crude            383
trade            368
interest         239
ship             154
money-supply     146
sugar            127
coffee           103
gnp               99
gold              96
veg-oil           70
cpi               68
oilseed           62
cocoa             50
copper            48
reserves          48
Name: Topic, dtype: int64

In [ ]:
df_test['Topic'].value_counts()[:20]

N/A             2917
earn             933
acq              642
crude            137
money-fx         124
grain            120
trade            101
interest          94
ship              45
money-supply      31
gnp               27
gold              25
sugar             24
veg-oil           24
coffee            21
alum              19
cocoa             17
oilseed           16
cpi               16
dlr               16
Name: Topic, dtype: int64

Since we are going to use a Supervised Algorithm, we are going to remove all the articles where no topic(<b>N/A</b>) has been assigned 

In [ ]:
print('Before Removing Articles without Topics: ',df_train.shape,df_test.shape)
df_train = df_train[df_train['Topic'] != 'N/A']
df_test = df_test[df_test['Topic'] != 'N/A']
print('After Removing Articles without Topics',df_train.shape,df_test.shape)

Before Removing Articles without Topics:  (15390, 2) (5610, 2)
After Removing Articles without Topics (8345, 2) (2693, 2)


In [ ]:
df_train.Topic.value_counts()[:20]

earn            2888
acq             1708
money-fx         530
grain            410
crude            383
trade            368
interest         239
ship             154
money-supply     146
sugar            127
coffee           103
gnp               99
gold              96
veg-oil           70
cpi               68
oilseed           62
cocoa             50
reserves          48
copper            48
livestock         47
Name: Topic, dtype: int64

We are going to consider the top 10 most ocurring categories in the train set and categorize remaining as  <b>'Other'</b> to do a multilabel classification of the news articles 

In [ ]:
topic_names = list(df_train['Topic'].value_counts()[:10].index)

In [ ]:
df_train['Topic'] = df_train['Topic'].apply(lambda x: x if x in topic_names else 'others')
df_test['Topic'] = df_test['Topic'].apply(lambda x: x if x in topic_names else 'others')

In [ ]:
df_train.Topic.value_counts()

earn            2888
acq             1708
others          1392
money-fx         530
grain            410
crude            383
trade            368
interest         239
ship             154
money-supply     146
sugar            127
Name: Topic, dtype: int64

Using <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html'>LabelEncoder</a> from sklearn to encode target variables between 0 and 10(number of topics - 1)

In [ ]:
le = LabelEncoder()
le.fit(df_train.Topic.unique())

LabelEncoder()

In [ ]:
le.classes_

array(['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx',
       'money-supply', 'others', 'ship', 'sugar', 'trade'], dtype=object)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Interos/label_encoder.pickle','wb') as f:
    pickle.dump(le,f)

Encoding our target variables

In [ ]:
df_train['labels'] = le.transform(df_train['Topic'])
df_test['labels'] = le.transform(df_test['Topic'])

In [ ]:
df_train.head()

,Article,Topic,labels
0,Showers continued throughout the week in the B...,others,7
4,The U.S. Agriculture Department reported the f...,grain,3
5,Argentine grain board figures show crop regist...,others,7
8,Champion Products Inc said its board of direct...,earn,2
9,Computer Terminal Systems Inc said it has comp...,acq,0


In [ ]:
df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

We will use the [Datasets](https://huggingface.co/docs/datasets/) library, and covert our Dataframe into that format using `from_pandas`function here.We are doing as it is faster to train on has various inbuilt functions for preprocessing it

In [ ]:
train_data=Dataset.from_pandas(df_train)

test_data = Dataset.from_pandas(df_test)

In [ ]:
train_data[:2]

{'Article': ['Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review.  The dry period means the temporao will be late this year.  Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures.  Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors.  There are doubts as to how much of this cocoa would be fit for expor

# Preprocessing

The first step is to use the DistilBERT tokenizer to first split the word into tokens. Then, we add the special tokens needed for sentence classifications (these are `[CLS]` at the first position, and `[SEP]` at the end of the sentence).
<img src="https://drive.google.com/uc?export=view&id=1GwAHbjMSUDBHKwMrUTSm4j6i3-9APpvG">

This is done by a Transformers [Tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html) which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
x = tokenizer(["Hello, this one sentence!",'This is another sentence, but a bit longer'],padding=True)

In [ ]:
print(x)

{'input_ids': [[101, 7592, 1010, 2023, 2028, 6251, 999, 102, 0, 0, 0], [101, 2023, 2003, 2178, 6251, 1010, 2021, 1037, 2978, 2936, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


<b>input_ids</b>: These are encoded ids assigned to the tokens(padded tokens have id=0)
<br>
<b>attention_mask</b>: This helps the model to know whether the input_id represent a token(attention_mask = 1) or padding(attention_mask=0)

In [ ]:
print(tokenizer.decode(x['input_ids'][0]))
print(tokenizer.decode(x['input_ids'][1]))

[CLS] hello, this one sentence! [SEP] [PAD] [PAD] [PAD]
[CLS] this is another sentence, but a bit longer [SEP]


As you can see above it adds a [CLS] and [SEP] token to the text as it tokenizes it and added [PAD] to make their lengths equal. This is done as the model accepts sequences of uniform length, so we add pad tokens to ake them uniform

In [ ]:
def preprocess_function(examples):
    """
    Function to be applied to Datasets objects to preprocess them.
    truncation=True, as the model only accepts maximum sequence length of 512 tokens
    and any sequence longer than that will be truncated.
    """
    return tokenizer(examples['Article'],truncation=True)

In [ ]:
encoded_train = train_data.map(preprocess_function,batched=True)
encoded_test = test_data.map(preprocess_function,batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
encoded_train

Dataset({
    features: ['Article', 'Topic', 'attention_mask', 'input_ids', 'labels'],
    num_rows: 8345
})

In [ ]:
# Dropping Reduntant Columns
encoded_train.remove_columns_(['Topic','Article'])
encoded_test.remove_columns_(['Topic','Article'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: remove_columns_ is deprecated and will be removed in the next major version of datasets. Use Dataset.remove_columns instead.
  


In [ ]:
metric = load_metric('glue','mnli')

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric)

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

Accuracy = (Number of Correct Observation) / (Number of Total Observation)


In [ ]:
fake_preds = np.random.randint(0, 11, size=(100,))
fake_labels = np.random.randint(0, 11, size=(100,))
print(sum(fake_preds == fake_labels)/100)
metric.compute(predictions=fake_preds, references=fake_labels)

0.12


{'accuracy': 0.12}

# Training the Model

Now that our data is ready, we can download the pretrained model and fine-tune it, as these models are trained on a huge corpus so we can use that our advantage to get better representation of words.

We are going to use the `DistilBERT` Model which is smaller version of `BERT` having 66 Million parameters as compared to 350 million parameters of BERT and having simillar accuracy on a large number of tasks.

We will first give the input to the DistilBERT model

<img src="https://drive.google.com/uc?export=view&id=18vxl1csb51wiGJWhjdcoYLBahQnPCRvQ">

Since this is a sentence classification task, we ignore all except the first vector (the one associated with the [CLS] token). The one vector we pass as the input to the Dense layer of the model.

This is how our final model looks like

<img src="https://drive.google.com/uc?export=view&id=1bWxXnoCUUNM6NfVH_OXSuq2ZYd2cFO_N">

We are using the [transformers](https://huggingface.co/transformers/) Library from Hugging Face

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 11
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

We will now define  the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    "/content/drive/MyDrive/Interos/",
    evaluation_strategy = "epoch",  # Evaluation to be done at the end of each epoch
    save_strategy = "epoch",  # Save model after every epoch so after the end of the trainig we can return the best one
    learning_rate=2e-5,     # Learning rate (alpha) for gradient descent
    per_device_train_batch_size=16, # batch_size = 16
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` as 16(DUe to colab constarint)  and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.


The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We are passing the tokenizer here again ,because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it.

Now we will train our Model


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 8345
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1566


Epoch,Training Loss,Validation Loss,Accuracy
1,0.584600,0.259691,0.928333
2,0.197300,0.222775,0.937245
3,0.123800,0.226701,0.940215


***** Running Evaluation *****
  Num examples = 2693
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Interos/checkpoint-522
Configuration saved in /content/drive/MyDrive/Interos/checkpoint-522/config.json
Model weights saved in /content/drive/MyDrive/Interos/checkpoint-522/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Interos/checkpoint-522/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Interos/checkpoint-522/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2693
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Interos/checkpoint-1044
Configuration saved in /content/drive/MyDrive/Interos/checkpoint-1044/config.json
Model weights saved in /content/drive/MyDrive/Interos/checkpoint-1044/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Interos/checkpoint-1044/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Interos/checkpoint-1044

TrainOutput(global_step=1566, training_loss=0.2953824735083647, metrics={'train_runtime': 2178.2501, 'train_samples_per_second': 11.493, 'train_steps_per_second': 0.719, 'total_flos': 3050572355883234.0, 'train_loss': 0.2953824735083647, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2693
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.9402153731897512,
 'eval_loss': 0.2267012894153595,
 'eval_runtime': 71.9306,
 'eval_samples_per_second': 37.439,
 'eval_steps_per_second': 2.349}

Above we have trained our model to classify the articles into one of 11 classes and achieved an accuracy of 94.02%.<br>
[Link](https://colab.research.google.com/drive/11xrw6bcOqIU8ocAzWzjSV3nVXQL7tv-j?usp=sharing) to the prediction notebook

In [ ]:
# Freeing up Resources
del trainer
torch.cuda.empty_cache()
torch.cuda.ipc_collect()